Read me!: 
1. If you want your data that you created to be saved after closing your notebook, you neeed to navigate to "Session options" on the right and select "Variables and files"
2. Enable the "Internet On" toggle so the notebook is able to download packages from the internet. Wihout this enbaled the next step won't work.
3. You need to select an accelarator in the dropdown menu. P100 is the fastest in my experience. Below you will see you much time you have left using this accelarator. You get 30Hours a week. The second you open a notebook with an accelarator these 30 hours are getting used up. 
4. Everytime you start the notebook you will need to start the code cell 1. because other than your dataset and output directory nothing will be saved.

# 1. Download all the needed and setup miniconda (This will take a while)

In [ ]:
!curl -L https://repo.anaconda.com/miniconda/Miniconda3-py39_4.12.0-Linux-x86_64.sh -o miniconda.sh
!chmod +x miniconda.sh
!sh miniconda.sh -b -p /content/miniconda
!/content/miniconda/bin/pip install --quiet acids-rave
!/content/miniconda/bin/pip install --quiet --upgrade ipython ipykernel
!/content/miniconda/bin/conda install -y ffmpeg

# Show helpfull hints about the training parameters in the console

In [ ]:
!/content/miniconda/bin/rave train --helpfull

# 3.Training

In [ ]:
# Training
name = "GiveYourModelAName" 
dataset = "/kaggle/input/YourDatateset" 
architecture = "v2" #["v3", "v2", "v1", "discrete", "onnx", "raspberry" ]
regularization = "wasserstein" # ["default", "wasserstein", "spherical"]
channels = 1 #Number of audio channels
val_every = 25000 #Specify after how many steps it should make a validation step where it creates an audio for you to listen to.
workers = 4 #This depends on the amount of threads your CPU has. Kaggle mostly has 4 so you can leave it as is.

%cd /kaggle/working #changes directory to kaggel/working
!mkdir -p preprocessed_dataset #if there isn't already a folder name preprocessed_dataset it will create one.

preprocessed_dataset = "/kaggle/working/preprocessed_dataset" #assigns preprocessed_dataset to the path in your kaggle/working directory

if regularization == "default":
  !/content/miniconda/bin/rave preprocess --input_path $dataset --output_path $preprocessed_dataset --channels $channels
  !/content/miniconda/bin/rave train --config $architecture --db_path $preprocessed_dataset --name $name --val_every $val_every --channels $channels --workers $workers
else:
  !/content/miniconda/bin/rave preprocess --input_path $dataset --output_path $preprocessed_dataset --channels $channels
  !/content/miniconda/bin/rave train --config $architecture --config $regularization --db_path $preprocessed_dataset --name $name --val_every $val_every --channels $channels --workers $workers

# Make an archive to download your training progress

In [ ]:
#Use this code cell when you want to make an archive to download it. 
# You can download the archive once it is done by clickig the little dots next to it and selecting download. 
# There is no download progress bar so you will only see the finished download pop up in youd download folder once it is done.
import shutil

# Specify the directory you want to archive
source_dir = '/kaggle/working/runs'

# Specify the name and format of the archive file
archive_name = '/kaggle/working/240213'
format = 'zip'  # You can use 'zip', 'tar', 'gztar', or 'bztar'

# Create the archive
shutil.make_archive(archive_name, format, source_dir)

# Remove a folder if necessary

In [ ]:
import shutil

# Specify the path of the folder you want to delete
folder_path = '/path/to/a/folder"'

# Use shutil.rmtree() to remove the folder and all its contents recursively
shutil.rmtree(folder_path)

# Delete file (useful if you want to delete a zip file you made earlier to clear space in your output folder)

In [ ]:
import os

# Specify the path to the file you want to delete
file_to_delete = '/kaggle/working/fileYouWantToDelete'

# Delete the file
os.remove(file_to_delete)

# 3. Resume Training

In [ ]:
# Resume Training
ckpt = "/kaggle/working/runs/YourModel/latestVersion/checkpoints" #Path to folder where your last checkpoint is saved.
name = "GiveYourModelAName" 
dataset = "/kaggle/input/YourDatateset" 
architecture = "v2" #["v3", "v2", "v1", "discrete", "onnx", "raspberry" ]
regularization = "wasserstein" # ["default", "wasserstein", "spherical"]
channels = 1 #Number of audio channels
val_every = 25000 #Specify after how many steps it should make a validation step where it creates an audio for you to listen to.
workers = 4 #This depends on the amount of threads your CPU has. Kaggle mostly has 4 so you can leave it as is.

%cd /kaggle/working
!mkdir -p preprocessed_dataset

preprocessed_dataset = "/kaggle/working/preprocessed_dataset"

if regularization == "default":
  !/content/miniconda/bin/rave train --config $architecture --db_path $preprocessed_dataset --name $name --ckpt $resume --val_every $val_every --channels $channels --workers $workers
else:
  !/content/miniconda/bin/rave train --config $architecture --config $regularization --db_path $preprocessed_dataset --name $name --ckpt $resume --val_every $val_every --channels $channels --workers $workers

# Output helpfull hints about the export paramets to the console

In [ ]:
!/content/miniconda/bin/rave export --helpfull

# 4. Export

In [ ]:
# Export as torchscript (.ts)
%cd /kaggle/working
!mkdir -p trained
model_dir = ""
# If rave is a Python package, install it with pip
!/content/miniconda/bin/rave export --run $model_dir --streaming 